In [ ]:
import numpy as np
import soundfile as sf
import librosa  

# Load the .wav file
data, samplerate = sf.read("ambisonic_audio.wav")

def compute_audio_at_direction(W: np.ndarray, X: np.ndarray, 
                                   Y: np.ndarray, Z: np.ndarray,
                                   theta: float, phi: float) -> float:
    """
    Compute audio waveform/energy at a specific spherical direction.
    Uses the spherical harmonic reconstruction formula:
    s(θ, φ, t) = W(t)Y_W(θ,φ) + X(t)Y_X(θ,φ) + Y(t)Y_Y(θ,φ) + Z(t)Y_Z(θ,φ)
    
    Args:
        W, X, Y, Z: Ambisonic channel waveforms for this time frame
        theta: Azimuth angle (longitude) in radians
        phi: Elevation angle (latitude) in radians
        
    Returns:
        energy: Audio energy at this direction
    """
    # Spherical harmonic basis functions (First-Order Ambisonics)
    # Using ACN/SN3D convention
    Y_W = 1.0                                    # Y_0^0 (omnidirectional)
    Y_X = np.cos(phi) * np.cos(theta)           # Y_1^1 (front-back)
    Y_Y = np.cos(phi) * np.sin(theta)           # Y_1^-1 (left-right)
    Y_Z = np.sin(phi)                            # Y_1^0 (up-down)
    
    # Reconstruct audio signal at this direction
    signal = W * Y_W + X * Y_X + Y * Y_Y + Z * Y_Z
    
    # Compute energy (RMS)
    energy = np.sqrt(np.mean(signal ** 2))
    
    return energy

longitude = 30
latitude = 67
    
print(f"Shape: {data.shape}")   # (num_samples, 4)
print(f"Sample rate: {samplerate} Hz")

# Split into channels
W = data[:, 0]
X = data[:, 1]
Y = data[:, 2]
Z = data[:, 3]


def processWave(type, wave, sampleRate):
    windowSize = 2048
    hopSize = 100
    
    # converts this to a Short-Time Fourier Transform. Tells you how much eergy has at each frequency over time.
    # does this by going through windows. Length of each window defined by n_fft. Then, shifts window to right by length
    # hop length. at each window, computes how much of each frequency is present.
    # final value is 2D array of rows being each frequency, columns being time (which is now the windows), so value being amplitude/energy for that time and frequency
    stftWave = np.abs(librosa.stft(wave, n_fft=windowSize, hop_length=hopSize))
    # when we get the mel, that just converts all the frequencies to 128 possible onces, which are moreso frequencies humans can hear. So compressing
    # the frequencies from a large number of frequencies to a smaller number, in this case n_mels amount
    mel = librosa.feature.melspectrogram(S = stftWave, sr= sampleRate, n_mels = 128)
    # converts from power scaling of audio to decibel scaling, cause humans perceive in moreso logarithm of audio (so higher sounds kinda taper off to us)
    logMel = librosa.power_to_db(mel, ref=np.max)

    # gets overall frame energy, including amplitude
    frameEnergy = np.sqrt(np.mean(logMel ** 2, axis=0))
    # gets the contrast in energy between frequencies within a specific frequency band, so where some frequencies bands may have parts of high energy frequencies, while other parts are low energy
    contrast = librosa.feature.spectral_contrast(S = stftWave, sr=sampleRate)
    # combines the difference frequency bands to get a average contrast for that time frame
    contrast = np.mean(contrast, axis=0)
    # basically gets how much the sound chagnes over time. Does this by getting differnece over time fimes with np.diff, squaring that value, and getting its sum
    temporal_novelty = np.sum(np.diff(logMel, axis=1) ** 2, axis=0)
    # do this to add an extra value cause rn, the length is T - 1, since you're getting difference between frames. So add 1 to get it to T length
    temporal_novelty = np.insert(temporal_novelty, 0, 0)

    # standard normalization
    frameEnergyNorm = frameEnergy / np.max(frameEnergy)
    contrastNorm = contrast / np.max(contrast)
    temporal_novelty_norm = temporal_novelty / np.max(temporal_novelty)

    # gets it for the overall time
    finalSaliency = 0.5 * frameEnergyNorm + 0.3 * contrastNorm + 0.2 * temporal_novelty_norm

    secondsIn = 6.5
    # divide by hoSize as first secondIn * sr gets the specific sample we want. Dividing by hopSize tells us how many windows to traverse
    # to get to that sample, as say if it's less than hopSize, its in the window at index 0, if it's slightly more, it's in the window at index 1, etc
    index = int(secondsIn * sampleRate / hopSize)


    print(f"For type of wave {type}, saliency was {finalSaliency[index]}")



processWave("Example", wave, samplerate)
